In [2]:
import numpy as np
import pandas as pd
import pymorphy2
from tqdm.notebook import tqdm as tn
import nltk
import regex as re
nltk.download('stopwords')
russian_stopwords = nltk.corpus.stopwords.words('russian')
english_stopwords = nltk.corpus.stopwords.words('english')
word_tokenizer = nltk.WordPunctTokenizer()
nltk.download('wordnet')
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
import json


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
data_spbu = pd.read_csv(r"C:\Users\User\Downloads\spbu_content.csv", header = None)
data_spbu.head(5)

,0,1,2
0,https://spbu.ru/,2023-04-14T00:56:50.6368499Z,Главная | Санкт-Петербургский государствен...
1,https://cabinet.spbu.ru/Account/LogOn/,2023-04-14T00:57:02.6226399Z,Вход ...
2,http://bb.spbu.ru/,2023-04-14T00:57:02.6225294Z,Blackboard Learn ...
3,https://events.spbu.ru/,2023-04-14T00:57:03.8827793Z,...
4,https://cabinet.spbu.ru/Account/Register/,2023-04-14T00:57:06.6414451Z,Регистраци...


In [4]:
data_msu = pd.read_csv(r"C:\Users\User\Downloads\msu_content.csv", header = None)
data_msu.head(5)

,0,1,2
0,https://msu.ru/,2023-04-15T16:29:55.1662631Z,\t\tМосковский государственный университет име...
1,https://msu.ru/news/rss/,2023-04-15T16:30:13.1464229Z,"\tНовости МГУhttp://www.msu.ruSat, 15 Apr 2023..."
2,http://270.msu.ru/,2023-04-15T16:30:09.1446391Z,270 лет Московскому университету Указ Пре...
3,https://conf.msu.ru/rus/event/8147/,2023-04-15T16:30:09.1695835Z,...
4,http://nosh.msu.ru/,2023-04-15T16:30:11.2455821Z,Научно-образовательные школы Московского унив...


In [8]:
import re
text = "dctv"
def has_cyrillic(text): # проверяем, слово содержит русские буквы или нет 
    return bool(re.search('[а-яА-Я]', text))

In [9]:
import re
text = "路"
def has_english(text): # проверяем, слово содержит английские буквы или нет 
    return bool(re.search('[a-zA-z]', text))

In [10]:
data_spbu = data_spbu[~data_spbu[0].str.endswith('.pdf/')] # удаляем строки из датафрейма, которые пдфки

In [6]:
morph = pymorphy2.MorphAnalyzer()


def lemmatization(data_spbu):
    df = data_spbu.copy()
    list_errors = list()
    for i in tn(data_spbu.index):
        if i < 30433:
            sub_string = data_spbu.iloc[i, 2]
        if type(sub_string) is str:
            word_list = nltk.word_tokenize(sub_string.lower()) # делаем маленькие буквы, разбиваем текст на токены (из 3-го столбика по каждой строке)
            word_list = [word for word in word_list if (word not in russian_stopwords and not word.isnumeric() and word not in english_stopwords and (has_cyrillic(word) or has_english(word)))] # удаляем слова, которые в англ и русс стоп-словах или если они числовые (строчки)
            try:
                word_list = [morph.parse(j)[0].normal_form for j in word_list] # нормальная форма токена
                df.iloc[i, 2] = ' '.join(word_list) # заменяем 3 столбик на лемматизированный текст
            except:
                list_errors.append(i) # в лист ошибок добавляем номер строки, в которой ошибка 
                df.iloc[i, 2] = '' # зануляем строку, в которой ошибка
    return df, list_errors 
df, list_errors = lemmatization(data_spbu) # переменные из ретерна, которые возвращает функция 
# df.to_csv('df.csv') # сохраняем обработанный датафрейм в новый csv для последующего использования (токенизированный и почищенный)

In [4]:
list_errors = [426, # лист с ошибками
3306,
4132,
7316,
8177,
8209,
8277,
8354,
8500,
8525,
8851,
9725,
9726,
9784,
9829,
9857,
9957,
9994,
10082,
10430,
10431,
10432,
10966,
11017,
11025,
11045,
11215,
11306,
11533,
11691,
11784,
12063,
12214,
12273,
12526,
12528,
12760,
12769,
12884,
12927,
12974,
14278,
15082,
16247,
16267,
16316,
16360,
16600,
16803,
17497,
17772,
18650,
18690,
18817,
20048,
20664,
21126,
21710,
22305,
22794,
23144,
23278,
23613,
23720,
23766,
23838,
23911,
24235,
24485,
25062,
25117,
25512,
25775,
26357,
27083,
27179,
27583,
29401,
29573,
29743,
30433]

In [14]:
morph = pymorphy2.MorphAnalyzer()
df = data_spbu.copy()


In [ ]:
# мне нужен словарь, в котором слова - ключи, а значения(которые соответствуют этим ключам - списки)
# то есть каждому слову соответствует список ссылок из df.csv, в которых содержится это слово
# 

In [8]:
df = pd.read_csv('df.csv', index_col=0)
list_errors = [426, 3306, 4132,7316, 8177, 8209, 8277, 8354, 8500, 8525, 8851, 9725, 9726, 9784, 9829, 9857, 9957, 9994, 10082, 10430, 10431, 
               10432, 10966, 11017, 11025, 11045, 11215, 11306, 11533, 11691, 11784, 12063, 12214, 12273, 12526, 12528, 12760, 12769, 12884,
               12927, 12974, 14278, 15082, 16247, 16267, 16316, 16360, 16600, 16803, 17497, 17772, 18650, 18690, 18817, 20048, 20664, 21126,
               21710, 22305, 22794, 23144, 23278, 23613, 23720, 23766, 23838, 23911, 24235, 24485, 25062, 25117, 25512, 25775, 26357, 27083, 
               27179, 27583, 29401, 29573, 29743, 30433] 
# list_errors = lemmatization(data_spbu)
def create_dict(df, list_errors):
    my_dict = {}
    df_size = df.shape[0]   
    for i in tn(df.index):  # цикл, который обрабатывает строки
        if i not in list_errors and isinstance(df.iloc[i][2], str) and i < df_size:
            tokens_list = list()
            tokens_list = df.iloc[i][2].split() # если строка не встречается в листе ошибок (то что не получилось обработать на лемматизации)
            for j in tokens_list: # тут цикл, который обрабатывает токены (он смотрит, есть ли какая-то запись в словаре, соответствующая конкретному слову)
                if j in my_dict: # если ключ уже есть, то добавляем в список ссылку
                    my_dict[j].append(df.iloc[i][0]) 
                else:  
                    my_dict[j] = [df.iloc[i][0]] # пыталась тут добавить ключ и значение (то есть ссылку), если их нет в словаре 

    return my_dict

my_dict = create_dict(df, list_errors)           
            # если запись в словаре есть - идем дальше, если записи в словаре нет - добавляем эту строку в лист для конкретного ключа
            # ну либо (если первый раз встречается) создаем лист, в который записывается значение строки для ключа

  0%|          | 0/30433 [00:00<?, ?it/s]

IndexError: single positional indexer is out-of-bounds

In [9]:
import json # сохраняем словарь в файл 
with open('result.json', 'w') as fp:
    json.dump(my_dict, fp)